In [1]:
#get vectorized yesterday ISW
#get last 24h weather
#add regions
#get alarms

In [2]:
import pandas as pd
import datetime
import weather_scrap as w
import alerts_in_ua as a

### Merging regions and weather

In [3]:
df_regions = pd.read_csv(f"../data/regions.csv")

In [4]:
df_regions.head(5)

,region,center_city_ua,center_city_en,region_alt,region_id
0,АР Крим,Сімферополь,Simferopol,Крим,1
1,Вінницька,Вінниця,Vinnytsia,Вінниччина,2
2,Волинська,Луцьк,Lutsk,Волинь,3
3,Дніпропетровська,Дніпро,Dnipro,Дніпропетровщина,4
4,Донецька,Донецьк,Donetsk,Донеччина,5


In [5]:
df_regions["region"] = df_regions["region"].apply(lambda x: x + " обл.")

In [6]:
weather = w.generate_forecast("Lviv, Ukraine")
df_weather = pd.DataFrame(weather)
df_weather.head(5)

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_uvindex,hour_conditions
0,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,0.0,[rain],27.4,9.0,201.7,1010.0,24.1,42.0,1.0,"Rain, Partially cloudy"
1,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,0.0,[rain],22.3,7.6,257.1,1012.0,10.6,72.4,0.0,"Rain, Partially cloudy"
2,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,0.0,[rain],23.0,5.0,267.9,1011.0,24.1,93.9,0.0,"Rain, Overcast"
3,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,0.0,[rain],17.6,7.2,270.5,1012.0,19.8,94.5,0.0,"Rain, Overcast"
4,"Львів, Україна",2025-04-19,1745010000,19.4,12.6,15.6,9.9,71.0,10.2,25.00,...,0.0,[rain],21.6,6.8,287.2,1012.0,24.1,100.0,0.0,"Rain, Overcast"


In [7]:
df_weather.columns

Index(['city_resolvedAddress', 'day_datetime', 'day_datetimeEpoch',
       'day_tempmax', 'day_tempmin', 'day_temp', 'day_dew', 'day_humidity',
       'day_precip', 'day_precipcover', 'day_solarradiation',
       'day_solarenergy', 'day_uvindex', 'day_sunrise', 'day_sunset',
       'day_moonphase', 'hour_datetime', 'hour_datetimeEpoch', 'hour_temp',
       'hour_humidity', 'hour_dew', 'hour_precip', 'hour_precipprob',
       'hour_snow', 'hour_snowdepth', 'hour_preciptype', 'hour_windgust',
       'hour_windspeed', 'hour_winddir', 'hour_pressure', 'hour_visibility',
       'hour_cloudcover', 'hour_uvindex', 'hour_conditions'],
      dtype='object')

In [8]:
df_weather["city"] = df_weather["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
df_weather["city"] = df_weather["city"].replace('Хмельницька область', "Хмельницький")

In [9]:
df_weather.head(5)

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_uvindex,hour_conditions,city
0,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,[rain],27.4,9.0,201.7,1010.0,24.1,42.0,1.0,"Rain, Partially cloudy",Львів
1,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,[rain],22.3,7.6,257.1,1012.0,10.6,72.4,0.0,"Rain, Partially cloudy",Львів
2,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,[rain],23.0,5.0,267.9,1011.0,24.1,93.9,0.0,"Rain, Overcast",Львів
3,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,[rain],17.6,7.2,270.5,1012.0,19.8,94.5,0.0,"Rain, Overcast",Львів
4,"Львів, Україна",2025-04-19,1745010000,19.4,12.6,15.6,9.9,71.0,10.2,25.00,...,[rain],21.6,6.8,287.2,1012.0,24.1,100.0,0.0,"Rain, Overcast",Львів


In [10]:
df_weather_reg = pd.merge(df_weather, df_regions, left_on="city",right_on="center_city_ua")

In [11]:
df_weather_reg.head(5)

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,hour_visibility,hour_cloudcover,hour_uvindex,hour_conditions,city,region,center_city_ua,center_city_en,region_alt,region_id
0,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,24.1,42.0,1.0,"Rain, Partially cloudy",Львів,Львівська обл.,Львів,Lviv,Львівщина,13
1,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,10.6,72.4,0.0,"Rain, Partially cloudy",Львів,Львівська обл.,Львів,Lviv,Львівщина,13
2,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,24.1,93.9,0.0,"Rain, Overcast",Львів,Львівська обл.,Львів,Lviv,Львівщина,13
3,"Львів, Україна",2025-04-18,1744923600,24.8,13.0,18.3,10.4,61.6,0.1,4.17,...,19.8,94.5,0.0,"Rain, Overcast",Львів,Львівська обл.,Львів,Lviv,Львівщина,13
4,"Львів, Україна",2025-04-19,1745010000,19.4,12.6,15.6,9.9,71.0,10.2,25.00,...,24.1,100.0,0.0,"Rain, Overcast",Львів,Львівська обл.,Львів,Lviv,Львівщина,13


In [12]:
df_weather_reg["region"] = df_weather_reg["region"].apply(lambda x: x + " область")

In [13]:
kyiv_reg = df_weather_reg[df_weather_reg["region"] == "Київська область"].copy()
kyiv_reg["region"] = "Київ"

df_weather_reg = pd.concat([df_weather_reg, kyiv_reg])

### Merging weather, alarms and regions together

In [14]:
df_alarms = pd.DataFrame(a.get_alerts())
df_alarms.head()

,region,status
0,Автономна Республіка Крим,A
1,Волинська область,N
2,Вінницька область,N
3,Дніпропетровська область,N
4,Донецька область,N


In [15]:
df_alarms["status"] = df_alarms["status"].apply(lambda x: 1 if x == 'A' else 0)

In [16]:
df_weather_reg_al = df_weather_reg.merge(df_alarms, how="left", left_on="region", right_on="region")

### Merging vectorized ISW to the rest

In [17]:
df_isw_vect = pd.read_csv("../data/ISW_vector.csv")

In [18]:
df_isw_vect.tail()

,date,content,lemma_content,stem_content,keywords
1077,2025-02-25,russian offensive campaign assessment 2025 chr...,russian offensive campaign assessment 2025 chr...,russian offens campaign assess 2025 christina ...,0.467 0.468 0.875 0.458 0.682 0.482 0.455 0.5 ...
1078,2025-02-26,russian offensive campaign assessment 2025 dav...,russian offensive campaign assessment 2025 dav...,russian offens campaign assess 2025 davit gasp...,0.436 0.828 0.441 0.674 0.473 0.463 0.476 0.45...
1079,2025-02-27,russian offensive campaign assessment 2025 chr...,russian offensive campaign assessment 2025 chr...,russian offens campaign assess 2025 christina ...,0.434 0.925 0.441 0.611 0.462 0.446 0.489 0.48...
1080,2025-02-28,russian offensive campaign assessment 2025 oli...,russian offensive campaign assessment 2025 oli...,russian offens campaign assess 2025 olivia gib...,0.506 0.478 0.865 0.768 0.493 0.522 0.496 0.49...
1081,2025-03-01,russian offensive campaign assessment 2025 dav...,russian offensive campaign assessment 2025 dav...,russian offens campaign assess 2025 davit gasp...,0.462 0.463 0.795 0.716 0.472 0.482 0.488 0.45...


In [30]:
df_isw_vect = df_isw_vect[df_isw_vect["date"] == datetime.date.today() - datetime.timedelta(days=1)]

KeyError: 'report_date'

In [ ]:
#df_ready = df_weather_reg_al.merge(df_isw, how="left", left_on="day_datetime", right_on="date_tomorrow_datetime")